In [2]:
import pandas as pd

columns_to_take = [
    "id",
    "sex",
    "age_category",
    "sample",
    "aging_rate",
    "Age",
    "Biological_Age",
    "Ethnicity.White",
    "Ethnicity.British",
    "Ethnicity.Irish",
    "Ethnicity.White_Other",
    "Ethnicity.Mixed",
    "Ethnicity.White_and_Black_Caribbean",
    "Ethnicity.White_and_Black_African",
    "Ethnicity.White_and_Asian",
    "Ethnicity.Mixed_Other",
    "Ethnicity.Asian",
    "Ethnicity.Indian",
    "Ethnicity.Pakistani",
    "Ethnicity.Bangladeshi",
    "Ethnicity.Asian_Other",
    "Ethnicity.Black",
    "Ethnicity.Caribbean",
    "Ethnicity.African",
    "Ethnicity.Black_Other",
    "Ethnicity.Chinese",
    "Ethnicity.Other",
    "Ethnicity.Other_ethnicity",
    "Ethnicity.Do_not_know",
    "Ethnicity.Prefer_not_to_answer",
    "Ethnicity.NA",
]


list_information = []

for chamber_type in [3, 4]:
    information_raw = pd.read_csv(
        f"../data/page12_AttentionMapsVideos/RawVideos/files/AttentionMaps-samples_Age_Heart_MRI_{chamber_type}chambersRawVideo.csv",
        usecols=columns_to_take,
    )[columns_to_take].set_index("id")
    information_raw.drop(index=information_raw[information_raw["aging_rate"] != "normal"].index, inplace=True)

    information = pd.DataFrame(
        None,
        columns=["chamber", "sex", "age_group", "sample", "chronological_age", "biological_age", "ethnicity"],
        index=information_raw.index,
    )

    information["chamber"] = chamber_type
    information["sex"] = information_raw["sex"]
    information["age_group"] = information_raw["age_category"]
    information["sample"] = information_raw["sample"]
    information["chronological_age"] = information_raw["Age"].astype(int)
    information["biological_age"] = information_raw["Biological_Age"]

    for id_participant in information_raw.index:
        ethnicities = information_raw.loc[
            id_participant, information_raw.columns[information_raw.columns.str.startswith("Ethnicity")]
        ]

        information.loc[id_participant, "ethnicity"] = " ".join(
            list(map(lambda list_ethni: list_ethni.split(".")[1], ethnicities[ethnicities == 1].index))
        )

    list_information.append(information.reset_index())

In [14]:
information_raw.drop(index=information_raw[information_raw["aging_rate"] != "normal"].index, inplace=True)

,sex,age_category,sample,aging_rate,Age,Biological_Age,Ethnicity.White,Ethnicity.British,Ethnicity.Irish,Ethnicity.White_Other,...,Ethnicity.Black,Ethnicity.Caribbean,Ethnicity.African,Ethnicity.Black_Other,Ethnicity.Chinese,Ethnicity.Other,Ethnicity.Other_ethnicity,Ethnicity.Do_not_know,Ethnicity.Prefer_not_to_answer,Ethnicity.NA
id,,,,,,,,,,,,,,,,,,,,,
2458643_2,Male,young,0,accelerated,53.256670,67.516934,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4462040_2,Male,young,1,accelerated,52.101300,66.151699,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5970635_2,Male,young,2,accelerated,50.004110,61.532150,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4836928_2,Male,young,3,accelerated,54.195755,65.533157,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3328720_2,Male,young,4,accelerated,52.791240,63.733083,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4131855_2,Female,old,5,decelerated,80.268310,69.563884,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1777223_2,Female,old,6,decelerated,74.661194,64.524168,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4669097_2,Female,old,7,decelerated,75.485280,65.390848,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
